In [11]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [12]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [13]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Choose important cases to generate, important scenario is the scenario that users are more likely to encounter it.
- Limit the appearance of scenarios that are hard to happen. 
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.

Example:
The scenario's name structure should start with Verify + who or what role can do this action + what they are testing in this scenario.
Verify that users can successfully create a new account.
Verify that users cannot create a new account if the username field is blank.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.

Example:
The scenario's name structure should start with Verify + who or what role can do this action + what they are testing in this scenario.
Verify that users can successfully create a new account.
Verify that users cannot create a new account if the username field is blank.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and use c"invalid": "below 8 characters, over 30 chaase specification.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.

Example:
The scenario's name structure should start with Verify + who or what role can do this action + what they are testing in this scenario.
Verify that users can successfully create a new account.
Verify that users cannot create a new account if the username field is blank.
"""


FILTER_SYSTEM_PROMPT="""
Your task is to read all of this scenarios generation from many sources.
Then you remove all the same copies because there are many duplicate scenarios in meanings inside the input.
If a scenario from the flow scenario is duplicate with a scenario from the condition scenario, delete the one from the flow scenario and keep the one from the condition.
Make sure every scenarios in the response is unique.
Return scenario name only.
"""

# Identify what type of that element (button,buttons, icon,scroll, text field, radio buttons, menu, menu for navigation,menu for opening dialog or another menu, menu for filter, sliders, switches, dialog, link, form,rating, filter).


In [14]:
usecase_name = "AddLesson"
project_name = "MatchaEnglishWebsite"
main_flow_prompt="""
basic flow:
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system open dropdown menu for Lesson with options: "Add new Lesson", "View list of Lesson", "Lesson statistics".
Step 3: Administrator selects "View list of Lesson" options.
Step 4: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 5: Administrator select button "Add new Lesson"
Step 6: The system redirects Admin to the "Add new Lesson" page.
Step 7: Administrator adds the name of the lesson in the name field.
Step 8: Administrator adds the description of the lesson in the description field.
Step 9: Administrator selects "Save" to create a new lesson.
"""

alt_prompt_1="""
basic flow:
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system open dropdown menu for Lesson with options: "Add new Lesson", "View list of Lesson", "Lesson statistics".
Step 3: Administrator selects "View list of Lesson" options.
Step 4: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 5: Administrator select button "Add new Lesson"
Step 6: The system redirects Admin to the "Add new Lesson" page.
Step 7: Administrator adds the name of the lesson in the name field.
Step 8: Administrator adds the description of the lesson in the description field.
Step 9: Administrator selects "Save" to create a new lesson.

Alternative flow 1: Administrator adds an already used name for the lesson.
At step 3 of the basic flow: Administrator adds a name that is already used in the existed lessons in the name field.
step 4: Administrator adds the description of the lesson in the description field.
step 5: Administrator selects "Save" to create a new lesson.
step 6: The system shows notification that the name is already used, prompting Administrator to add another name in the name field.
step 7: Administrator adds a new name in the name field.
step 8: Administrator selects "Save" to create a new lesson.
"""

alt_prompt_2="""
basic flow:
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system open dropdown menu for Lesson with options: "Add new Lesson", "View list of Lesson", "Lesson statistics".
Step 3: Administrator selects "View list of Lesson" options.
Step 4: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 5: Administrator select button "Add new Lesson"
Step 6: The system redirects Admin to the "Add new Lesson" page.
Step 7: Administrator adds the name of the lesson in the name field.
Step 8: Administrator adds the description of the lesson in the description field.
Step 9: Administrator selects "Save" to create a new lesson.

Alternative flow 2: Navigate to Add new lesson page by options in navbar. 
At step 3 of the basic flow: Administrator selects "Add new Lesson".
Go back to step 6 in the basic flow and continue with the steps from step 6.
"""

alt_prompt_3="""
basic flow:
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system open dropdown menu for Lesson with options: "Add new Lesson", "View list of Lesson", "Lesson statistics".
Step 3: Administrator selects "View list of Lesson" options.
Step 4: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 5: Administrator select button "Add new Lesson"
Step 6: The system redirects Admin to the "Add new Lesson" page.
Step 7: Administrator adds the name of the lesson in the name field.
Step 8: Administrator adds the description of the lesson in the description field.
Step 9: Administrator selects "Save" to create a new lesson.

Exception flow:
At step 3 of the basic flow: Administrator can select "Cancel" to stop adding a new lesson at anytime.
"""

prompt_all="""
basic flow:
Step 1: Admin presses the tab "Lesson Management" on the navbar.
Step 2: The system open dropdown menu for Lesson with options: "Add new Lesson", "View list of Lesson", "Lesson statistics".
Step 3: Administrator selects "View list of Lesson" options.
Step 4: The system redirects Admin to the Lesson Management page which show lesson list which admins created.
Step 5: Administrator select button "Add new Lesson"
Step 6: The system redirects Admin to the "Add new Lesson" page.
Step 7: Administrator adds the name of the lesson in the name field.
Step 8: Administrator adds the description of the lesson in the description field.
Step 9: Administrator selects "Save" to create a new lesson.

Alternative flow 1: Administrator adds an already used name for the lesson.
At step 3 of the basic flow: Administrator adds a name that is already used in the existed lessons in the name field.
step 4: Administrator adds the description of the lesson in the description field.
step 5: Administrator selects "Save" to create a new lesson.
step 6: The system shows notification that the name is already used, prompting Administrator to add another name in the name field.
step 7: Administrator adds a new name in the name field.
step 8: Administrator selects "Save" to create a new lesson.

Alternative flow 2: Navigate to Add new lesson page by options in navbar. 
At step 3 of the basic flow: Administrator selects "Add new Lesson".
Go back to step 6 in the basic flow and continue with the steps from step 6.

Exception flow:
At step 3 of the basic flow: Administrator can select "Cancel" to stop adding a new lesson at anytime.
"""

In [15]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

promptSubScenario3 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_3}
  ]
sub3_gpt_response = ask(promptSubScenario3, client, model)
filtercontent = "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response + "\n" + sub2_gpt_response + "\n" + sub3_gpt_response


promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)


filtercontent += "\n" + "Conditional scenario:" + "\n" + condition_scenario_response
print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("\n" + "Final result:" + filter_gpt_response)

{'Name field': {'condition': {'valid': 'Administrator adds the name of the lesson, name is not already used', 'invalid': 'Administrator adds an already used name for the lesson'}, 'type': 'text field'}, 'Description field': {'condition': {'valid': 'Administrator adds the description of the lesson', 'invalid': ''}, 'type': 'text area'}}
Verify that Administrator cannot create a new lesson with an already used name.
Flow scenario:
Verify that Admin can successfully create a new lesson.
Verify that Admin cannot create a new lesson if the name field is blank.
Verify that Admin cannot create a new lesson if the description field is blank.
Verify that Admin receives a confirmation message after successfully creating a new lesson.
Verify that Admin is redirected to the correct page after selecting "View list of Lesson".
Verify that Admin can access the "Add new Lesson" page from the Lesson Management page.
Verify that administrators can successfully create a new lesson.
Verify that administra

In [16]:
print(filter_gpt_response)

Verify that Admin can successfully create a new lesson.
Verify that Admin cannot create a new lesson if the name field is blank.
Verify that Admin cannot create a new lesson if the description field is blank.
Verify that Admin receives a confirmation message after successfully creating a new lesson.
Verify that Admin is redirected to the correct page after selecting "View list of Lesson".
Verify that Admin can access the "Add new Lesson" page from the Lesson Management page.
Verify that administrators can navigate to the "Add new Lesson" page via the dropdown menu in the navbar.
Verify that administrators can cancel the lesson creation process.
Verify that Administrator cannot create a new lesson with an already used name.


In [17]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

promptSubScenario3 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_3}
  ]
sub3_gpt_response = ask(promptSubScenario3, client, model)
filtercontent = "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response + "\n" + sub2_gpt_response + "\n" + sub3_gpt_response


promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)


filtercontent += "\n" + "Conditional scenario:" + "\n" + condition_scenario_response
print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("\n" + "Final result:" + filter_gpt_response)

{'Name field': {'condition': {'valid': 'Name entered is not already used', 'invalid': 'Name entered is already used'}, 'type': 'text field'}, 'Description field': {'condition': {'valid': 'Description is entered', 'invalid': 'Description is not entered'}, 'type': 'text area'}}
Verify that administrators cannot create a new lesson if the lesson name is already used.
Verify that administrators cannot create a new lesson if the description field is not entered.
Flow scenario:
Verify that Admin can successfully create a new lesson.
Verify that Admin cannot create a new lesson if the name field is blank.
Verify that Admin cannot create a new lesson if the description field is blank.
Verify that Admin can view the list of lessons they created.
Verify that Admin can access the "Lesson statistics" option.
Verify that administrators can add a new lesson with a unique name.
Verify that administrators cannot add a new lesson with an already used name.
Verify that administrators can navigate to the

In [18]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

promptSubScenario3 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_3}
  ]
sub3_gpt_response = ask(promptSubScenario3, client, model)
filtercontent = "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response + "\n" + sub2_gpt_response + "\n" + sub3_gpt_response


promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)


filtercontent += "\n" + "Conditional scenario:" + "\n" + condition_scenario_response
print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("\n" + "Final result:" + filter_gpt_response)

{'Name field': {'condition': {'valid': 'Administrator adds a unique name', 'invalid': 'Name is already used'}, 'type': 'text field'}, 'Description field': {'condition': {'valid': 'Administrator adds the description', 'invalid': 'Description is not added'}, 'type': 'text area'}}
Verify that administrators cannot create a new lesson if the name field is already used.
Verify that administrators cannot create a new lesson if the description field is not added.
Flow scenario:
Verify that Admin can successfully create a new lesson.
Verify that Admin cannot create a new lesson if the name field is blank.
Verify that Admin cannot create a new lesson if the description field is blank.
Verify that Admin receives confirmation after successfully creating a new lesson.
Verify that Admin is redirected to the correct page after selecting "Add new Lesson" from the dropdown menu.
Verify that Admin can view the list of lessons created by them.
Verify that administrators can add a new lesson with a uniqu

In [19]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

promptSubScenario3 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_3}
  ]
sub3_gpt_response = ask(promptSubScenario3, client, model)
filtercontent = "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response + "\n" + sub2_gpt_response + "\n" + sub3_gpt_response


promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)


filtercontent += "\n" + "Conditional scenario:" + "\n" + condition_scenario_response
print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("\n" + "Final result:" + filter_gpt_response)

{'Name field': {'condition': {'valid': 'name is entered and not already used', 'invalid': 'name field is empty or name is already used'}, 'type': 'text field'}, 'Description field': {'condition': {'valid': 'description is entered', 'invalid': 'description field is empty'}, 'type': 'text area'}}
Verify that administrators cannot add a new lesson if the name field is empty.
Verify that administrators cannot add a new lesson if the name is already used.
Verify that administrators cannot add a new lesson if the description field is empty.
Flow scenario:
Verify that Admin can successfully create a new lesson.
Verify that Admin cannot create a new lesson if the name field is blank.
Verify that Admin cannot create a new lesson if the description field is blank.
Verify that Admin receives confirmation after successfully creating a new lesson.
Verify that Admin is redirected to the correct page after selecting "Add new Lesson" from the Lesson Management dropdown.
Verify that Admin can view the 

In [20]:
promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

promptSubScenario2 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_2}
  ]
sub2_gpt_response = ask(promptSubScenario2, client, model)

promptSubScenario3 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_3}
  ]
sub3_gpt_response = ask(promptSubScenario3, client, model)
filtercontent = "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response + "\n" + sub2_gpt_response + "\n" + sub3_gpt_response


promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)


filtercontent += "\n" + "Conditional scenario:" + "\n" + condition_scenario_response
print(filtercontent)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("\n" + "Final result:" + filter_gpt_response)

{'Name field': {'condition': {'valid': 'Administrator enters a unique name', 'invalid': 'Name is already used, empty'}, 'type': 'text field'}, 'Description field': {'condition': {'valid': 'Administrator adds description', 'invalid': 'Empty'}, 'type': 'text area'}}
Verify that Administrator cannot add a new lesson if the name field is already used.
Verify that Administrator cannot add a new lesson if the name field is empty.
Verify that Administrator cannot add a new lesson if the description field is empty.
Flow scenario:
Verify that Admin can successfully create a new lesson.
Verify that Admin cannot create a new lesson if the name field is blank.
Verify that Admin cannot create a new lesson if the description field is blank.
Verify that Admin is redirected to the Lesson Management page after selecting "View list of Lesson".
Verify that Admin is redirected to the "Add new Lesson" page after selecting "Add new Lesson" button.
Verify that Admin can view the list of lessons they created.